# Função de Hash

In [1]:
import hashlib

def sha256(payload):
    return hashlib.sha256(payload).hexdigest()

# Curva Elíptica

In [2]:
NIST = dict()
NIST['P-192'] =  {
    'p': 6277101735386680763835789423207666416083908700390324961279,
    'n': 6277101735386680763835789423176059013767194773182842284081,
    'seed' : '3045ae6fc8422f64ed579528d38120eae12196d5',
    'c': '3099d2bbbfcb2538542dcd5fb078b6ef5f3d6fe2c745de65',
    'b': '64210519e59c80e70fa7e9ab72243049feb8deecc146b9b1',
    'Gx' : '188da80eb03090f67cbf20eb43a18800f4ff0afd82ff1012',
    'Gy' : '07192b95ffc8da78631011ed6b24cdd573f977a11e794811'
}

NIST['P-256'] =  {
    'p': 115792089210356248762697446949407573530086143415290314195533631308867097853951,
    'n': 115792089210356248762697446949407573529996955224135760342422259061068512044369 ,
    'seed' : 'c49d360886e704936a6678e1139d26b7819f7e90',
    'c': '7efba1662985be9403cb055c75d4f7e0ce8d84a9c5114abcaf3177680104fa0d',
    'b': '5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b',
    'Gx' : '6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296',
    'Gy' : '4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5'
}

USED_NONCES = []

# Classe que implementa o ECDSA

In [3]:
class ECDSA:
    def __init__(self):
        self.E, self.q, self.N, self.G = self.setup()
        
    def setup(self):
        c = NIST['P-256']
        p  = c['p'] 
        n  = c['n']
        b  = ZZ(c['b'],16)
        Gx = ZZ(c['Gx'],16)
        Gy = ZZ(c['Gy'],16)
        E = EllipticCurve(GF(p),[-3,b])
        q = E.order()
        N = GF(q) # Zq
        G = E((Gx,Gy))
        return E, q, N, G
    
    def gen_key_pair(self):
        d = ZZ(self.N.random_element()) # private key
        Q = d*self.G                    # public key
        return d, Q
    
    def sign(self, m, d):
        r = 0; s = 0; N = self.N; G = self.G; q = self.q
        m = ZZ(sha256(message),16)
        while r == s == 0:
            k = ZZ.random_element(1, q)
            if not k in USED_NONCES:
                r = N((k*G).xy()[0])
                s = N((m+d*r)/N(k))
            USED_NONCES.append(k)
        return r, s
    
    def verify(self, m, r, s, Q):
        m = ZZ(sha256(message),16)
        w = 1/s
        ri = ZZ(r*w)
        mi = ZZ(m*w)
        if r == self.N((mi*self.G + ri*Q).xy()[0]):
            print 'Valid signature'
        else:
            print 'Invalid signature'

# Exemplo de utilização

In [4]:
ecdsa = ECDSA()

# Generate key pair
private_key, public_key = ecdsa.gen_key_pair()

# Message
message = "exemplo de uma mensagem!"

# Sign
ephemeral_key, signature = ecdsa.sign(message, private_key)

# Verify
ecdsa.verify(message, ephemeral_key, signature, public_key)

Valid signature
